In [ ]:
#https://huggingface.co/blog/davidberenstein1957/fine-tune-modernbert-on-synthetic-data

In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers==4.48.0
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
import datasets

from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
import transformers
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import evaluate
import pandas as pd
import numpy as np



In [4]:
import sklearn
from sklearn.model_selection import train_test_split
dataFile = '/content/drive/MyDrive/mafiaWHistory.tsv'
df=pd.read_csv(dataFile, sep='\t')
print(df)

df = df.drop(["game", "Unnamed: 0.1", "Unnamed: 0", "message", "withName", "history","mId", "role","name", "time" ], axis=1)
#df['msgAndHistory'] = df.apply(dataFix, axis = 1)
df = df.rename(columns={'label': 'labels','msgAndHistory': 'text'})
print(df)

train, validate = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)


mtrain = Dataset.from_pandas(train)
meval = Dataset.from_pandas(validate)

      Unnamed: 0.1  Unnamed: 0  mId             name       role  \
0                0         431    2   Natalie Morris      mafia   
1                1         422    3      Ryan Hodges      mafia   
2                2         432    4   Natalie Morris      mafia   
3                3         423    5      Ryan Hodges      mafia   
4                4         413   10    Mary Trujillo  bystander   
...            ...         ...  ...              ...        ...   
2202          2202          53   98    Nicole Barker  bystander   
2203          2203          67  101    Angela Murphy  bystander   
2204          2204          68  102    Angela Murphy  bystander   
2205          2205          69  104  Richard Stevens      mafia   
2206          2206          70  105  Jessica Michael      mafia   

                                                message label  \
0                                                    Yo     D   
1                                                    hi     D   


In [6]:
d = df.loc[df["labels"] == "D"]
print(d)

     labels                                               text
0         D  Game history: "" Detect deception for the foll...
1         D  Game history: "Natalie Morris: Yo" Detect dece...
2         D  Game history: "Natalie Morris: Yo Ryan Hodges:...
3         D  Game history: "Natalie Morris: Yo Ryan Hodges:...
7         D  Game history: "Natalie Morris: Yo Ryan Hodges:...
...     ...                                                ...
2187      D  Game history: "Jessica Michael: hey Richard St...
2192      D  Game history: "Jessica Michael: hey Richard St...
2198      D  Game history: "Jessica Michael: hey Richard St...
2205      D  Game history: "Jessica Michael: hey Richard St...
2206      D  Game history: "Jessica Michael: hey Richard St...

[813 rows x 2 columns]


In [7]:

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
#def tokenize(batch):

#    return tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")

# Tokenize dataset
#if "label" in split_dataset["train"].features.keys():
#    split_dataset =  split_dataset.rename_column("label", "labels") # to match Trainer


def preprocess_function(examples):
    label_mapping = {"T": 0, "D": 1}

    res  = tokenizer(examples["text"], padding=True, truncation=True, max_length=512)
    res['labels'] = [label_mapping[label] for label in examples['labels']]

    return res

tokenized_train = mtrain.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_eval = meval.map(preprocess_function, batched=True, remove_columns=["text"])

#tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
print(tokenized_train)

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2096
})


In [ ]:
print(transformers.__version__)

4.48.0


In [9]:

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
labels = ["T", "D"]#tokenized_dataset["train"].features["labels"].names


num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
#crashes if not 4.48
# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    pred = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=pred, references=labels)


In [11]:
!pip install 'accelerate>=0.26.0'

In [12]:
from transformers import EarlyStoppingCallback


In [13]:
# Define training args
training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/models/mbertMafiawhistory",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=15,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    #use_mps_device=True,
    metric_for_best_model="accuracy",
    report_to="none",

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]

)


In [14]:
trainer.train()

W0507 09:18:57.217000 797 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.659700,0.558215,0.684685
2,0.460800,0.306441,0.864865
3,0.305500,0.164886,0.936937
4,0.128700,0.163422,0.909910
5,0.140600,0.121193,0.918919
6,0.091900,0.119260,0.927928
7,0.098000,0.123941,0.927928
8,0.102100,0.156622,0.918919


TrainOutput(global_step=1048, training_loss=0.2227062919667659, metrics={'train_runtime': 2960.9828, 'train_samples_per_second': 10.618, 'train_steps_per_second': 0.664, 'total_flos': 5713827561996288.0, 'train_loss': 0.2227062919667659, 'epoch': 8.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.16488602757453918,
 'eval_accuracy': 0.9369369369369369,
 'eval_runtime': 6.4377,
 'eval_samples_per_second': 17.242,
 'eval_steps_per_second': 1.087,
 'epoch': 8.0}